In [1]:
import os
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import ta
import requests
from dotenv import load_dotenv
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler


In [2]:
def fetch_stock_data(stock_name, interval="1h", period="6mo"):  
    stock = yf.Ticker(stock_name)
    df = stock.history(period=period, interval=interval) 
    
    df = df[["Open", "High", "Low", "Close", "Volume"]].dropna()

    return df

stock_name = "AAPL"  # Change stock symbol as needed
stock_data = fetch_stock_data(stock_name)

In [3]:
stock_data["RSI"] = ta.momentum.RSIIndicator(stock_data["Close"], window=14).rsi()

macd = ta.trend.MACD(stock_data["Close"])
stock_data["MACD"] = macd.macd()
stock_data["Signal_Line"] = macd.macd_signal()

bb = ta.volatility.BollingerBands(stock_data["Close"], window=20, window_dev=2)
stock_data["Upper_Band"] = bb.bollinger_hband()
stock_data["Lower_Band"] = bb.bollinger_lband()

stock_data["VWAP"] = ta.volume.VolumeWeightedAveragePrice(
    stock_data["High"], stock_data["Low"], stock_data["Close"], stock_data["Volume"]
).volume_weighted_average_price()

adx = ta.trend.ADXIndicator(stock_data["High"], stock_data["Low"], stock_data["Close"])
stock_data["ADX"] = adx.adx()

stock_data.dropna(inplace=True)



In [4]:
required_features = ["Close", "RSI", "MACD", "VWAP", "ADX"]

missing_features = [col for col in required_features if col not in stock_data.columns]
if missing_features:
    print(f"❌ Error: Missing columns {missing_features}")
else:
    print(f"✅ All required features are present!")

print(stock_data.isnull().sum())

✅ All required features are present!
Open           0
High           0
Low            0
Close          0
Volume         0
RSI            0
MACD           0
Signal_Line    0
Upper_Band     0
Lower_Band     0
VWAP           0
ADX            0
dtype: int64


In [11]:
import requests
from nltk.sentiment import SentimentIntensityAnalyzer

def get_stock_news_sentiment_vader(stock_name):
    api_key = "084eb76cbfac4a0ba55fc1a33880b441"  
    url = f"https://newsapi.org/v2/everything?q={stock_name}&apiKey={api_key}"
    
    response = requests.get(url).json()

    if "articles" not in response or not response["articles"]:
        return "No recent news available.", "Neutral"

    latest_news_title = response["articles"][0]["title"] 
    sia = SentimentIntensityAnalyzer()
    
    sentiment_score = sia.polarity_scores(latest_news_title)['compound']

    if sentiment_score > 0.05:
        sentiment = "Positive"
    elif sentiment_score < -0.05:
        sentiment = "Negative"
    else:
        sentiment = "Neutral"

    return latest_news_title, sentiment

# Example 1
stock_name = "TSLA"  # Tesla stock
news, sentiment = get_stock_news_sentiment_vader(stock_name)

print(f"Latest News: {news}")
print(f"Market Sentiment: {sentiment}")

Latest News: Elon Musk makes controversial decision that could cost Tesla big time
Market Sentiment: Negative


In [12]:
# Example 2
stock_name = "AAPL"  # google stock
news, sentiment = get_stock_news_sentiment_vader(stock_name)

print(f"Latest News: {news}")
print(f"Market Sentiment: {sentiment}")

Latest News: AAPL opens market at highest price since Trump tariff bombshell
Market Sentiment: Neutral


In [13]:
scaler = MinMaxScaler(feature_range=(0,1))
stock_data_scaled = scaler.fit_transform(stock_data[["Close", "RSI", "MACD", "VWAP", "ADX"]])

scaled_df = pd.DataFrame(stock_data_scaled, columns=["Close", "RSI", "MACD", "VWAP", "ADX"], index=stock_data.index)

In [14]:
selected_features = ["Close", "RSI", "MACD", "VWAP", "ADX"]
X = scaled_df[selected_features].values
y = scaled_df["Close"].shift(-1).dropna().values  

X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(X[:-1], y, test_size=0.2, random_state=42)

print("X_train_xgb shape:", X_train_xgb.shape)
print("y_train_xgb shape:", y_train_xgb.shape)

xgb_model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100)
xgb_model.fit(X_train_xgb, y_train_xgb)

X_train_xgb shape: (660, 5)
y_train_xgb shape: (660,)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [15]:
xgb_model.save_model("xgb_model.json")

In [ ]:
from tensorflow.keras.models import load_model
import xgboost as xgb

# XGBoost model
try:
    xgb_model = xgb.XGBRegressor()
    xgb_model.load_model("xgb_model.json")  
    print("XGBoost model loaded successfully!")
except Exception as e:
    print(f"Error loading XGBoost model: {e}")


In [ ]:
feature_scaler = MinMaxScaler(feature_range=(0,1))
close_scaler = MinMaxScaler(feature_range=(0,1))

stock_data_scaled = feature_scaler.fit_transform(stock_data[["Close", "RSI", "MACD", "VWAP", "ADX"]])
stock_data["Close_Scaled"] = close_scaler.fit_transform(stock_data[["Close"]])


In [ ]:
def predict_next_hours(stock_data, hours=1):
    required_features = ["Close", "RSI", "MACD", "VWAP", "ADX"]

    missing_features = [col for col in required_features if col not in stock_data.columns]
    if missing_features:
        print(f"Error: Missing columns {missing_features}")
        print("Available columns:", stock_data.columns)
        return None

    if xgb_model is None:
        print("Error: XGBoost model is not loaded!")
        return None

    stock_data.fillna(0, inplace=True)

    try:
        last_window = stock_data[required_features].values[-1].reshape(1, -1)

        predicted_values = []
        for _ in range(hours):
            xgb_pred_scaled = xgb_model.predict(last_window)[0]
            final_pred_original = close_scaler.inverse_transform([[xgb_pred_scaled]])[0][0]
            predicted_values.append(final_pred_original)

        return predicted_values[-1]  

    except Exception as e:
        print(f"⚠️ Error in prediction: {e}")
        return None


In [ ]:
print("Last 5 Close Prices:", stock_data["Close"].tail(5).values)

selected_hours = 3 
predicted_price = predict_next_hours(stock_data, selected_hours)

print(f"Predicted Price in {selected_hours} Hours: {predicted_price}")


In [ ]:
for hours in [1, 3, 7]:
    predicted_price = predict_next_hours(stock_data, hours)
    print(f"Predicted Price in {hours} Hours: {predicted_price}")


In [ ]:
import mplfinance as mpf
import matplotlib.pyplot as plt

mpf.plot(stock_data.tail(100), type='candle', style='charles', title=f"{stock_name} Stock Chart")

plt.axhline(y=predicted_price, color='r', linestyle='--', label=f"Predicted Price in {selected_hours}H")
plt.legend()
plt.show()


In [ ]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor()
xgb_model.load_model("xgb_model.json")

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred_xgb = xgb_model.predict(X_test_xgb)
mse_xgb = mean_squared_error(y_test_xgb, y_pred_xgb)
r2_xgb = r2_score(y_test_xgb, y_pred_xgb)

print(f"XGBoost Model Performance:")
print(f"Mean Squared Error: {mse_xgb}")
print(f"R² Score: {r2_xgb:.2f} (Closer to 1 is better)")


In [ ]:
from tensorflow.keras.models import load_model

model = load_model("lstm_model.keras", compile=False)

model.compile(optimizer="rmsprop", loss="mean_squared_error")

print("Model loaded and recompiled successfully!")


In [ ]:
import os

if os.path.exists("xgb_model.json"):
    xgb_model = xgb.XGBRegressor()
    xgb_model.load_model("xgb_model.json")
    print("Model loaded successfully!")
else:
    print("Error: xgb_model.json not found! Train and save the model first.")


In [ ]:
import os
print(os.getcwd())  
print(os.listdir())

In [ ]:
xgb_model = xgb.XGBRegressor()
xgb_model.load_model("xgb_model.json")

close_scaler = MinMaxScaler(feature_range=(0, 1))

def fetch_stock_data(stock_name, interval="1h", period="6mo"):
    """Fetch stock data using Yahoo Finance API"""
    stock = yf.Ticker(stock_name)
    df = stock.history(period=period, interval=interval)
    
    if df.empty:
        print("❌ Error: Invalid Stock Name or No Data Found!")
        return None
    
    df = df[["Open", "High", "Low", "Close", "Volume"]].dropna()

    return df

def preprocess_data(stock_data):
    """Compute technical indicators and normalize data"""
    stock_data["RSI"] = ta.momentum.RSIIndicator(stock_data["Close"], window=14).rsi()
    
    macd = ta.trend.MACD(stock_data["Close"])
    stock_data["MACD"] = macd.macd()

    stock_data["VWAP"] = ta.volume.VolumeWeightedAveragePrice(
        stock_data["High"], stock_data["Low"], stock_data["Close"], stock_data["Volume"]
    ).volume_weighted_average_price()

    adx = ta.trend.ADXIndicator(stock_data["High"], stock_data["Low"], stock_data["Close"])
    stock_data["ADX"] = adx.adx()

    stock_data.dropna(inplace=True)
    
    features = ["Close", "RSI", "MACD", "VWAP", "ADX"]
    stock_data_scaled = MinMaxScaler(feature_range=(0, 1)).fit_transform(stock_data[features])
    
    scaled_df = pd.DataFrame(stock_data_scaled, columns=features, index=stock_data.index)
   
    stock_data["Close_Scaled"] = close_scaler.fit_transform(stock_data[["Close"]])
    
    return scaled_df

def predict_stock_price(stock_data):
    """Predict stock price using XGBoost model"""
    required_features = ["Close", "RSI", "MACD", "VWAP", "ADX"]
    if any(col not in stock_data.columns for col in required_features):
        print("❌ Error: Missing required features!")
        return None

    last_window = stock_data[required_features].values[-1].reshape(1, -1)

    predicted_price_scaled = xgb_model.predict(last_window)[0]
    predicted_price = close_scaler.inverse_transform([[predicted_price_scaled]])[0][0]

    return predicted_price

stock_name = input("Enter the stock symbol (e.g., AAPL, TSLA, GOOG): ").strip().upper()
stock_data = fetch_stock_data(stock_name)

if stock_data is not None:
    stock_data = preprocess_data(stock_data)

    predicted_price = predict_stock_price(stock_data)
    print(f"\n Predicted Price of {stock_name} in the next 3 hours: {predicted_price:.2f}")